In [17]:
import numpy as np
import modin.pandas as pd
import plotly.graph_objects as go
import time
import matplotlib.pyplot as plt
from distributed import Client
from datetime import datetime, timedelta
from glob import glob
from os import path
from astropy import coordinates as coord
from astropy import units as u
from astropy.time import Time
from astropy.coordinates import Angle
import cartopy.crs as ccrs
client = Client()

D:\ProgramData\Anaconda3\lib\site-packages\distributed\node.py:160: UserWarning:

Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 3036 instead

D:\ProgramData\Anaconda3\lib\site-packages\distributed\client.py:1105: VersionMismatchWarning:

Mismatched versions found

+-------------+-----------+-----------+-----------+
| Package     | client    | scheduler | workers   |
+-------------+-----------+-----------+-----------+
| dask        | 2021.08.1 | 2021.08.1 | 2021.09.1 |
| distributed | 2021.08.1 | 2021.08.1 | 2021.09.1 |
+-------------+-----------+-----------+-----------+



In [6]:
def file_product4specific_days(start_date, end_date, product_flags, path_dir, subfolder_path_format):
    """ This is a function to find all files with certain product_flag from the start_date to end_date from path_dir
        Input Args:
            start_date: date from
            end_date: date to
            product_flags: certain files
            path_dir: files should be found in the path_dir
            subfolder_path_format: the sub-folder format
        Output Args:
            output_paths: filenames from start_date to end_date including product_flag
    """
    DateFormat = "%Y-%m-%d"
    start_date = datetime.strptime(start_date, DateFormat).date()
    end_date = datetime.strptime(end_date, DateFormat).date()

    delta_one_day = timedelta(days=1)
    date = start_date
    output_paths = {}
    for _, product_flag in enumerate(product_flags):
        file_paths = []
        while date <= end_date:
            subfolder_path = date.strftime(subfolder_path_format)
            data_folder = path.join(path_dir, subfolder_path)
            if path.isdir(data_folder):
                for filename in glob(path.join(data_folder, '*.txt')):
                    if product_flag in filename:
                        file_paths.append(filename)
            date += delta_one_day
        date = start_date
        output_paths[product_flag] = file_paths

    return output_paths

In [12]:
def xyz2latlon(date, secofday, xyz):
    """ This is a function to transfer a set of coordinates in cartesian coordinate of ITRS to those
        corresponding latitude and longitude.

    Args:
        date (str): specifically the starting date of this formulism
        secofday (int): second of the day which is exactly the [date]. Note that [secofday] can be 
                        more than 86400
        xyz (np.float): cartesian coordinates for each epoch
    """
    # if (secofday.__len__() != xyz.__len__()):
    #     raise ValueError("The dimensions of secofday and xyz are not compatible")
    
    time_stamps = np.asarray([], dtype=str)
    for _, sec in enumerate(secofday):
        sec = np.float64(sec)
        if sec > 86400.:
            date_1 = datetime.strptime(date, "%Y-%m-%d")
            date = date_1 + timedelta(days=1)
        time_stamps = np.append(
            time_stamps, date + ' ' + str(timedelta(seconds=sec)))
    xs = xyz[:, 0]
    ys = xyz[:, 1]
    zs = xyz[:, 2]
    ground_lat = np.asarray([])
    ground_lon = np.asarray([])
    for index, sec in enumerate(secofday):
        now = Time(time_stamps[index])
        cartrep = coord.CartesianRepresentation(
            x=xs[sec],
            y=ys[sec],
            z=zs[sec],
            unit=u.m
        )
        itrs = coord.ITRS(cartrep, obstime=now)
        loc = coord.EarthLocation(*itrs.cartesian.xyz)
        ground_lat = np.append(ground_lat, Angle(loc.lat).degree)
        ground_lon = np.append(ground_lon, Angle(loc.lon).degree)
    
    return ground_lat, ground_lon


In [18]:
start_date = '2018-12-01'
end_date = '2018-12-01'
files_20181201220181204 = file_product4specific_days(
    start_date, end_date, ['KBR1B', 'GNV1B'], r'..//..//..//..//gracefo_dataset', r"gracefo_1B_%Y-%m-%d_RL04.ascii.noLRI")

# read in files
kbr1b_x = pd.concat(map(lambda file: pd.read_csv(file,
                                                 skiprows=162,
                                                 names=['gps_time', 'biased_range', 'range_rate', 'range_accl', 'iono_corr', 'lighttime_err', 'lighttime_rate',
                                                        'lighttime_accl', 'ant_centr_corr', 'ant_centr_rate', 'ant_centr_accl', 'k_a_snr', 'ka_a_snr', 'k_b_snr',
                                                        'ka_b_snr', 'qualflg'],
                                                 dtype=np.float64,
                                                 sep='\s+'),
                        files_20181201220181204['KBR1B']),
                    ignore_index=True)
gnv1b_c = pd.concat(map(lambda file: pd.read_csv(file,
                                                 skiprows=148,
                                                 names=['gps_time', 'id', 'coord_ref',
                                                        'xpos', 'ypos', 'zpos', 'xpos_err', 'ypos_err', 'zpos_err',
                                                        'xvel', 'yvel', 'zvel', 'xvel_err', 'yvel_err', 'zvel_err',
                                                        'qualflg'],
                                                 sep='\s+'),
                        [ctemp for ctemp in files_20181201220181204['GNV1B'] if '_C_' in ctemp]),
                    ignore_index=True)
gnv1b_d = pd.concat(map(lambda file: pd.read_csv(file,
                                                 skiprows=148,
                                                 names=['gps_time', 'id', 'coord_ref',
                                                        'xpos', 'ypos', 'zpos', 'xpos_err', 'ypos_err', 'zpos_err',
                                                        'xvel', 'yvel', 'zvel', 'xvel_err', 'yvel_err', 'zvel_err',
                                                        'qualflg'],
                                                 sep='\s+'),
                        [ctemp for ctemp in files_20181201220181204['GNV1B'] if '_D_' in ctemp]),
                    ignore_index=True)

# convert xyz to lat lon
kbr1b_x['lat'], kbr1b_x['lon'] = xyz2latlon(start_date,
                                            np.floor(gnv1b_c.gps_time.to_numpy() - gnv1b_c.gps_time.to_numpy()[0]).astype(np.int64)[::5],
                                            np.c_[gnv1b_c.xpos.to_numpy(),
                                                  gnv1b_c.ypos.to_numpy(),
                                                  gnv1b_c.zpos.to_numpy()])

# ionosphere correction to TEC
f_ka = 32e9 #  unit in Hz
kbr1b_x['TEC'] = -kbr1b_x['iono_corr'] / 40.3 * f_ka**2

ax = plt.axes(projection=ccrs.Robinson())

ax.set_global()
lon2d, lat2d = np.meshgrid(kbr1b_x.lon.compute(), kbr1b_x.lat.compute())

plt.contourf(lon2d, lat2d, kbr1b_x.TEC.compute(), transform=ccrs.PlateCarree())

ax.coastlines()
ax.gridlines()

plt.show()

# fig = go.Figure(data=go.Scatter(
#         x=kbr1b_x.gps_time,
#         y=kbr1b_x.TEC,
#         mode='lines',
#     ))
# fig.update_layout()
# fig.show()
